# Archdaily爬虫

#### ——爬取archdaily搜索页下所有项目的大图，以及项目信息                                                     

### 设置URL

In [ ]:
#拆分 'https://www.archdaily.com/search/projects/categories/train-station'
#url='https://www.archdaily.com'
#url_search='/search/projects/categories/train-station'
#以下需要设置：

url='https://www.archdaily.com'
url_search='/search/projects/categories/train-station'

### 设置搜索页数

In [ ]:
num_of_pages = 3

### 设置输出目录路径

In [ ]:
out_dir='./archdaily'

## 设置好以上之后，点 Cell - Run All

In [ ]:
import requests
import socket
import urllib
import re
import os
import pandas as pd
import bs4
from bs4 import BeautifulSoup

In [ ]:
def get_links(text_search_page):
    result=re.findall('<a class="afd-search-list__link" href="(.*?)">',text_search_page,re.S)
    return result

In [ ]:
project_links=[]
for i in range(num_of_pages):
    if i < 30:
    continue
    url_page=url+url_search + '?page=' + str(i+1)
    #print(url_page)
    r = requests.get(url_page)
    print(r.status_code)
#状态码200表示请求成功
    text=r.text
    project_links+=get_links(text)

print(project_links)      

In [ ]:
def get_soup(link):
    url_project = url + link
    r = requests.get(url_project)
    ht=r.text
    soup = BeautifulSoup(ht , 'lxml')
    return soup

In [ ]:
#从项目页获取gallery页url
def get_photos_pageURL(link):
    url_project = url + link
    r = requests.get(url_project)
    text=r.text
    #result = re.findall("<a class='js-image-size__link.*?' href='(.*?)'.*?style",text,re.S)
    #result = re.findall("<a class='js-image-size__link.*?href='(/.*?)'.*?style",text,re.S)
    #result = re.findall("<a class='js-image-size__link.*?href='"+link+"(.*?).*?style",text,re.S)
    result = re.findall("href='"+link+"(.*?-jpg|.*?-photo|.*?-image)",text,re.S)
    url_photos = url_project + result[0]
    print('photosURL: ' + url_photos)
    return url_photos


In [ ]:
photos_links=[]

In [ ]:
i=1
while(i < len(project_links)+1):
    link=project_links[i-1]
    print(i)
    i+=1
    try:
        url_photos=get_photos_pageURL(link)
        photos_links.append(url_photos)
    except BaseException:
        continue
#print(photos_links)

In [ ]:
print(photos_links)

In [ ]:
df=pd.DataFrame()
i=1
while(i<len(project_links)+1):
    print(i)
    link=project_links[i-1]
    i+=1
    soup=get_soup(link)
    
    project_name=soup.h1.string.strip().split('/')[0]
    print(project_name)
    info=['name']
    value=[project_name]

    h3s=soup.find_all(name='h3',class_='afd-char-title')
    h3s=list(set(h3s)
            )
    for j in range(len(h3s)):
        info.append(h3s[j].string.strip())
        contents=h3s[j].next_sibling.next_sibling.contents
        temp=''
        for c in contents:
            if(isinstance(c,bs4.element.NavigableString)):
                temp += c.strip()
            elif(isinstance(c,bs4.element.Tag)):
                temp += c.string.strip()
        value.append(temp)
    
    dict1 = dict(zip(info,value))
    df=df.append(dict1,ignore_index=True)

df[:10]

In [ ]:
if(not os.path.exists(out_dir)):
    os.mkdir(out_dir)

In [ ]:
df.to_csv(out_dir+'/out.csv',encoding='utf-8')

In [ ]:
j=1 #断点，起始项目序号

In [ ]:
while(j<len(photos_links)+1):
#    if(j==2):
#        break
    url_photos=photos_links[j-1]
    print(url_photos)
    
    try:
        r = requests.get(url_photos)
    except BaseException:
        continue
        
    #print(r.status_code)
    text=r.text
    
    try:
        result=re.findall("url_large&quot;:&quot;(.*?)&quot;,&quot;url_slideshow",text)
    except BaseException:
        continue
    
    path = out_dir+'/'+str(j)
    if(not os.path.exists(path)):
        os.mkdir(path)
    
    i=1 #断点，当前项目起始图片序号
    for pic in result:
        try:
            res=re.search("jpg/(.*?)\.jpg",pic)    
            name=res.group(1)
            r = requests.get(pic)
            print(str(i)+'.jpg')
            with open(path+'/'+str(i)+'.jpg', 'wb')as fp:
                fp.write(r.content)
        except BaseException:
            continue
        i+=1
    j+=1